In [1]:
# !pip3 install git+https://github.com/rongardF/tvdatafeed.git@e6f6aaa7de439ac6e454d9b26d2760ded8dc4923
from tvDatafeed import TvDatafeed, Interval
from helpers_db import get_engine, run_sql
engine = get_engine()

In [35]:
from helpers_db import insert_on_conflict_nothing_ticks, truncate_table

tv = TvDatafeed()

def fetch_ticks(symbol, exchange, bars=10000):
  df = tv.get_hist(
      symbol=symbol,
      exchange=exchange,
      interval=Interval.in_1_hour,
      n_bars=bars,
      extended_session=True,
  )
  return df

def fix_symbol(eachange_symbol):
  return eachange_symbol.split(':')[-1] if ':' in eachange_symbol else eachange_symbol

def normalize_dataframe(df):
  df.reset_index(inplace=True)
  df['symbol'] = df['symbol'].apply(fix_symbol)
  if 'datetime' in list(df.columns.values):
    df['dt'] = df['datetime']
    df.drop(columns=['datetime'], inplace=True)
  return df

In [52]:
asset     = {'symbol': 'SURG', 'exchange': 'NASDAQ'} #NYSE NASDAQ
symbol    = asset['symbol']
exchange  = asset['exchange']
print(f"Processing {exchange}:{symbol}")
try:
  df = fetch_ticks(symbol, exchange)
  print(f"  count: {df.shape[0]}")
  df = normalize_dataframe(df)
  df.to_sql('ticks', engine, if_exists='append', index=False, method=insert_on_conflict_nothing_ticks)
except Exception as e:
  print(f"Exception {exchange}:{symbol}")
  print(str(e))

Processing NASDAQ:SURG
  count: 1235
